# **Visualització de l'evolució d'Europa envers dades socials, demogràfiques i socials**

## *Requisits d'execució*

In [1]:
#!pip3 install pyspark
#!pip3 install duckdb
import pyspark

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, mean, when, lit, count
from pyspark.sql.window import Window
from pyspark.sql.functions import mean
from pyspark.sql.functions import mean
from pyspark.sql.types import IntegerType, DoubleType

import pandas as pd
import matplotlib.pyplot as plt
import duckdb

In [ ]:
!git clone https://github.com/OscarMoliina/betterlifebetterhealth.git

Cloning into 'betterlifebetterhealth'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 302 (delta 45), reused 102 (delta 29), pack-reused 155
Receiving objects: 100% (302/302), 98.71 MiB | 26.38 MiB/s, done.
Resolving deltas: 100% (95/95), done.


## *Lectura de la base de dades*

In [ ]:
spark = SparkSession.builder \
    .appName("Preprocessing") \
    .config("spark.jars", "/content/betterlifebetterhealth/src/utils/duckdb.jar") \
    .getOrCreate()

result = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:/content/betterlifebetterhealth/data/db/exploitation_zone.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("dbtable", "join_table") \
    .load()

In [ ]:
result.columns

['Country',
 'Year',
 'Area_Km2',
 'CBR',
 'CDR',
 'Deaths',
 'E0',
 'Medage',
 'MR0_4',
 'Pop_Dens',
 'GSCA',
 'Schizophrenia (%)',
 'Bipolar disorder (%)',
 'Eating disorders (%)',
 'Anxiety disorders (%)',
 'Drug use disorders (%)',
 'Depression (%)',
 'Alcohol use disorders (%)',
 'Total population',
 'Population density, pers per sq km',
 'Total population, male (%)',
 'Total population, female (%)',
 'Mean age of women at birth of first child',
 'Women in the Labour Force, Percent of corresponding total for both sexes',
 'Female tertiary students, percent of total',
 'Female legislators, senior officials and managers, percent of total',
 'Female professionals, percent of total for both sexes',
 'Female clerks, percent of total for both sexes',
 'Female craft and related workers, percent of total for both sexes',
 'Female plant and machine operators and assemblers, percent of total for both sexes',
 'Female members of parliament, percent of total',
 'Total employment, growth rate'

## *Anàlisi visual de les variables representatives*

    En aquesta secció experimentarem l'evolució de les diferents variables presents a la base de dades. Farem ús d'eines visuals amb l'objectiu
    de poder analitzar-les de manera més senzilla.

In [ ]:
import plotly.express as px

# Variables d'interès
variables_of_interest = {
    'E0': 'Expectativa de vida en néixer',
    'Medage': 'Edat mitjana de la població',
    'MR0_4': 'Taxa de mortalitat infantil',
    'Female members of parliament, percent of total': 'Dones membres del parlament, percentatge del total',
    'Economic activity rate': 'Taxa d\'activitat econòmica'
}

# Definim els països d'interés
european_countries = [
    'Lithuania', 'Slovenia', 'Austria', 'Greece', 'Slovakia', 'Cyprus',
    'Estonia', 'Ireland', 'Switzerland', 'Belgium', 'Romania',
    'Sweden', 'Spain', 'Luxembourg', 'Hungary', 'Norway', 'Denmark',
    'Montenegro', 'Ukraine', 'Italy', 'Turkey', 'Germany', 'Bulgaria',
    'Bosnia and Herzegovina', 'Serbia', 'Poland', 'France', 'Latvia',
    'United Kingdom', 'Iceland', 'Croatia', 'Albania', 'Netherlands',
    'Portugal', 'Malta', 'Finland', 'Belarus'
]

# Filtrem la base de dades pels països seleccionats
df_europe = df[df['Country'].isin(european_countries)]

# Per cada variable
for variable, title in variables_of_interest.items():
    # Calculem els valors màxim i mínim amb l'objectiu d'establir una llegenda fixa
    max_value = df_europe[variable].max()
    min_value = df_europe[variable].min()

    # Generar el mapa de cloroplet per Europa
    fig = px.choropleth(df_europe,
                        locations="Country",
                        locationmode='country names',
                        color=variable,
                        hover_name="Country",
                        animation_frame="Year",
                        color_continuous_scale=px.colors.sequential.Plasma,
                        title=f'Evolució de {title} a Europa',
                        range_color=[min_value, max_value])

    fig.update_layout(geo=dict(scope='europe'))

    fig.update_traces(colorbar=dict(
        title=title,
        x=0.9,
        len=0.5,
        y=0.5,
        lenmode="fraction",
        thickness=20,
        ypad=50
    ))

    fig.update_layout(
        geo=dict(scope='europe'),
        width=2000,  # Amplada de la figura
        height=700   # Alçada de la figura
    )

    fig.show()
